In [22]:
#前処理後のindex作るための処理
import os
import elasticsearch as es
import math
import numpy as np
from itertools import chain
import sys
import tensorflow as tf

ES_HOST = "localhost"
escon = es.Elasticsearch(ES_HOST, port=9200)
GACRP_HOME=os.environ['GACRP_HOME']

In [23]:
data=[]
ans = []

# 元データ取得
es_size=10000
es_s_time='2m'
body = {
    "_source":["channelGrouping","visitNumber","isMobile","country","hits","bounces","newVisits","source","medium","isTrueDirect","transactionRevenue"],
    "query": {
      "match_all": {}
    }
  }
  
res=escon.search(index='gacrp_index',body=body,scroll=es_s_time,size=es_size)
es_s_id=res['_scroll_id']
es_s_size=len(res["hits"]["hits"])


In [24]:
def setByScroll(res,data,ans):
    for i in range(len(res["hits"]["hits"])):
        row=res["hits"]["hits"][i]["_source"]
        rec=[]

        if row['channelGrouping']=='Display':
            col_channelGrouping=1
        elif row['channelGrouping']=='Social':
            col_channelGrouping=2
        else:
            col_channelGrouping=0
        oneHot_col_channelGrouping=np.eye(2+1)[col_channelGrouping]
        
        col_visitNumber=min(int(row['visitNumber']),5)/5.0 

        if row['isMobile']=="TRUE":
            col_isMobile=1
        else:
            col_isMobile=0

        if row['country']=="United States":
            col_country =1
        elif row['country']=="Canada":
            col_country =2
        elif row['country']=="Japan":
            col_country=3
        else:
            col_country=0
        oneHot_col_country=np.eye(3+1)[col_country]
    
        col_hits=math.log10(int(row['hits'])*1.0)
    
        if row['bounces']=="1":
            col_bounces=1
        else:
            col_bounces=0

        if row['newVisits']== "1":
            col_newVisits=1
        else:
            col_newVisits=0
    
        if row['source']=="mail.googleplex.com" or row['source']=="dfa":
            col_source =1
        elif row['source']=="(direct)":
            col_source=2
        else:
            col_source=0
        oneHot_col_source=np.eye(2+1)[col_source]
        
        if row['medium']=="cpm":
            col_medium=1
        else:
            col_medium=0
        oneHot_col_medium=np.eye(1+1)[col_medium]
    
        if row['isTrueDirect']=="1":
            col_isTrueDirect=1
        else:
            col_isTrueDirect=0
    
        revenue = math.log(int(row['transactionRevenue'])+1)
    
        l=[oneHot_col_channelGrouping,[col_visitNumber],[col_isMobile],oneHot_col_country,[col_hits],[col_bounces],[col_newVisits],oneHot_col_source,oneHot_col_medium,[col_isTrueDirect]]
        data.append(list(chain.from_iterable(l)))
        ans.append(revenue)
    
    return(data,ans)

In [26]:
while(es_s_size>0):
    data,ans=setByScroll(res,data,ans)
    res=escon.scroll(scroll_id=es_s_id,scroll=es_s_time)
    es_s_id=res['_scroll_id']
    es_s_size=len(res["hits"]["hits"])
    #print(len(data),len(ans))
    

In [27]:
#print(data[0],ans[0])
#inputData=tf.transpose(data)
num_elements=18
num_rows=len(data)
inputData=np.reshape(data,(num_rows,num_elements))
#print("data",data[0])
#print("inputData",inputData[0],inputData.shape)
inputAns=np.reshape(ans,(num_rows,1))
#print("AnsShape",inputAns.shape)
#inputAns=ans
x =tf.placeholder("float",[None,num_elements])
y_=tf.placeholder("float",[None,1])

#a=tf.Variable( tf.ones([18,1]),name="weights")
a=tf.Variable( tf.random_uniform([18,1],-1.0,1.0),name="weights")    
#b=tf.Variable(tf.zeros([1,1]),name="intercept")
#b=tf.Variable(tf.random_normal([1]),name="intercept")
print(x.shape)
#y=tf.matmul(tf.transpose(a),x)+b
#y=tf.matmul(a,x)+b
y=tf.matmul(a,x)

init = tf.global_variables_initializer()
loss = tf.reduce_sum(tf.square(y_-y))
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(loss)


(?, 18)


In [28]:
print(len(inputAns))
print(inputAns[0:1])

903653
[[ 0.]]


In [29]:
with tf.Session() as sess:
    sess.run(init)
    print('初期状態')
    print('誤差' + str(sess.run(loss, feed_dict={x: inputData[0:1], y_: inputAns[0:1]})))
    #print("weigths: %f, intercept: %f" % (sess.run(a), sess.run(b) ))
    print("input:",inputAns[step+1:step+1+1])
    print("weigths:",sess.run(a),"intercept:",sess.run(b) )
    
    for step in range(200):
        sess.run(train_step, feed_dict={x: inputData[step+1:step+1+1], y_: inputAns[step+1:step+1+1]})
        if (step+1) % 20 == 0:
            print('\nStep: %s' % (step+1))
            print("input:",inputAns[step+1:step+1+1])
            print('誤差' + str(sess.run(loss, feed_dict={x: inputData[step+1:step+1+1], y_: inputAns[step+1:step+1+1]})))
            #print("weights:",sum(sess.run(a)), " intercept: %f" % ( sess.run(b)))
            print("weights:",sess.run(a), " intercept: %f" % ( sess.run(b)))
            


初期状態
誤差47.6068
('input:', array([[ 0.]]))
('weigths:', array([[ 0.4205029 ],
       [ 0.79467559],
       [ 0.9546659 ],
       [ 0.95973969],
       [ 0.81892395],
       [-0.80905747],
       [ 0.88751245],
       [-0.67791939],
       [ 0.0500319 ],
       [-0.90240216],
       [-0.18336964],
       [-0.82331133],
       [ 0.06766295],
       [-0.52988243],
       [ 0.31625152],
       [ 0.47546649],
       [-0.01074219],
       [ 0.72718287]], dtype=float32), 'intercept:', array([ 0.02560825], dtype=float32))

Step: 20
('input:', array([[ 0.]]))
誤差0.306742
('weights:', array([[ 0.04023584],
       [ 0.07603855],
       [ 0.09134723],
       [ 0.09183272],
       [ 0.07835879],
       [-0.0774147 ],
       [ 0.08492165],
       [-0.06486674],
       [ 0.0047873 ],
       [-0.08634638],
       [-0.01754573],
       [-0.07877859],
       [ 0.00647433],
       [-0.05070183],
       [ 0.03026054],
       [ 0.04549503],
       [-0.00102787],
       [ 0.06958052]], dtype=float32), ' inter